# Electric Vehicle Charge Scheduling MDP

### Dependencies

You will need to install POMDPs and POMDPToolbox locally before being able to run this notebook. This can be done by running the following in your local Julia 1.0 REPL: 
    - Pkg.add("POMDPs")
    - Pkg.add("POMDPToolbox")

In [1]:
using POMDPs, POMDPToolbox

┌ Warning: Importing POMDPToolbox. As of Julia 0.7 POMDPToolbox has been discontinued. Please use POMDPModelTools, POMDPPolicies, BeliefUpdaters, and/or POMDPSimulators instead.
└ @ POMDPToolbox C:\Users\Emily\.julia\packages\POMDPToolbox\OdZy7\src\POMDPToolbox.jl:4


### State Structure
Define state structure / make initial constructor

In [2]:
mutable struct evState
    p::Vector{Bool} # array of whether cars are present
    c::Vector{Int64} # array of charge in each car
    renew::Int64 # renewable energy level
    t::Int64 # time
    done::Bool # are we in a terminal state
end

# initial state constructor
evState(p,c,renew::Int64, renew::Int64, t::Int64) = evState(p,c,renew,t,false)

ErrorException: syntax: function argument names not unique

### MDP Structure
Define MDP structure with everything you would need / make initial constructor

In [3]:
struct evMDP <: MDP{evState,Vector{Bool}} 
    n::Int64 # number of cars
    T::Int64 # number of timesteps
    renew_levels::Int64 # number of renewable mixture levels, 0:renew_levels
    charge_levels::Int64 # number of charge levels, 0:charge_levels
    λ::Float64 # terminal reward weighting
end

# we use key worded arguments so we can change any of the values we pass in 
function evMDP(;n::Int64 = 3 # number of cars
                T::Int64 = 6 # number of timesteps
                renew_levels::Int64 = 4, # number of renewable mixture levels, 0:renew_levels
                charge_levels::Int64 = 4, # number of charge levels, 0:charge_levels
                λ::Float64 = 100) #terminal reward weighting
    return evMDP(n, T, renew_levels, charge_levels, λ)
end


LoadError: syntax: missing comma or ) in argument list

### States
Define all possible states

In [4]:
# convert number to an array of numbers using requested base system, array length
function num2array(number,base,array_length)
    base==2 ? finalarray = zeros(Bool, array_length) : finalarray = zeros(Int64, array_length)
    idx=1
    while number > 0
        finalarray[idx] = rem(number,base)
        number = div(number,base)
        idx+=1
    end
    return finalarray
end

function POMDPs.states(mdp::evMDP)
    s = GridWorldState[] # initialize an array of GridWorldStates
    
    # add every possible state. This includes every possible combination of present/charge array
    
    for iP = 0:(2^mdp.n-1)
        present = num2array(iP,2,mdp.n)
        
        for iC = 0:((mdp.charge_levels+1)^mdp.n-1)
            charge = num2array(iC,(mdp.charge_levels+1),mdp.n)
            
            for rl=0:mdp.renew_levels, t=1:mdp.T
            
                # if in final time, make sure the done flag is set on
                t==mdp.T ? push!(s,evState(present, charge, rl, t, true)) : push!(s,evState(present, charge, rl, t)) 
            end
        end
    end
    return s
end

### Actions
Define all possible action vectors

In [5]:
function POMDPs.actions(mdp::evMDP)
    # initialize empty action space a
    a = []    
    # populate with all combinations of actions, ex [true, false, true, true]
    for iA=0:(2^mdp.n-1)
        push!(a,num2array(iA,2,mdp.n))
    end
    return a
end
    

### Reward Function
Define the reward function

In [6]:
function POMDPs.reward(mdp::evMDP, state::evState, action::Vector{Bool}, statep::evState)
    r = mdp.lambda*mdp.charge_levels 
    if state.done
        max_c = mdp.charge_levels
        for i in 1:length(state.p)
            if state.p[i] == true
                r -= exp((max_c-state.c[i])/max_c)
            end
        end
    end
    return r
end

### Transition Function
Define the next-state transition probabilities (this is the hard one)

In [7]:
function POMDPs.transition(mdp::evMDP, state::evState, action::Vector{Bool})
end

### Miscellaneous Functions
Define other functions that POMDPs.jl needs

In [8]:
POMDPs.n_states(mdp::evMDP) = 2^mdp.n*(mdp.charge_levels+1)^mdp.n*(mdp.renew_levels+1)*mdp.T
POMDPs.n_actions(mdp::evMDP) = 2^mdp.n
POMDPs.discount(mdp::evMDP) = 1
POMDPs.isterminal(mdp::evMDP, s::evState) = s.done

In [26]:
# define state and action indexing
function indVal(base, a)
    ind = 1
    for i in 1:length(a)
        ind += a[i]*base^(i-1)
    end
    return ind
end    

function POMDPs.stateindex(mdp::evMDP, state::evState)
    indP = indVal(2,state.p)
    indC = indVal(mdp.charge_levels,state.c)
    indR = state.renew + 1
    indT = state.t
    maxP = 2^(mdp.n)
    maxC = (mdp.charge_levels + 1)^(mdp.n)
    maxR = mdp.renew_levels + 1
    maxT = mdp.T
    sInd = indP + (maxP*(indC-1)) + (maxC*maxP*(indR-1)) + (maxR*maxC*maxP*(indT-1))
    return sInd
end

function POMDPs.actionindex(mdp::evMDP, act::Vector{Bool})
    return indVal(2,act)
end

In [30]:
mdp1 = evMDP(3,2,5,5,0.1)
state1 = evState([1,0,0],[1,0,0],0,1,false)

println(POMDPs.stateindex(mdp1,state1))


2
2
1
1
10


### Implement Solvers / Simulators

In [ ]:
# first let's load the value iteration module
using DiscreteValueIteration

# initialize the problem
mdp = evMDP()

# initialize the solver
# max_iterations: maximum number of iterations value iteration runs for (default is 100)
# belres: the value of Bellman residual used in the solver (defualt is 1e-3)
solver = ValueIterationSolver(max_iterations=100, belres=1e-3)

# initialize the policy by passing in your problem
policy = ValueIterationPolicy(mdp) 

# solve for an optimal policy
# if verbose=false, the text output will be supressed (false by default)
solve(solver, mdp, policy, verbose=true);

### Simulations

In [ ]:
mdp = evMDP()
mdp.tprob=1.0
sim(mdp, evState(4,1), max_steps=10) do s
    println("state is: $s")
    a = :right
    println("moving $a")
    return a
end;